In [ ]:
import os
from remap import remap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
from mpl_toolkits.axes_grid.inset_locator import inset_axes
import geopandas as gpd

from cpt_convert import loadCPT
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
shp = gpd.read_file('./shapefiles/estadosl_2007.shp')
shp = shp[shp['SIGLAUF3'] == 'SP']
bbox = shp.bounds.values[0]

channel = files[0][19:21]
year = files[0][27:31]
day = files[0][31:34]
hour = files[0][34:36]
minute = files[0][36:38]
plt.figure(figsize=(17, 10))

plt.title('GOES-16 ABI CANAL %s - %sº dia de %s às %s:%s'%(channel, day, year, hour, minute))
b = Basemap(projection='merc', llcrnrlon=bbox[0], llcrnrlat=bbox[1], 
            urcrnrlon=bbox[2], urcrnrlat=bbox[3])
b.readshapefile('./shapefiles/estadosl_2007', name='sp', linewidth=2, color='red')

bbox = [-53.08059692, -25.25489044, -44.20065689, -19.78015137]
# bbox = [-60, -33, -45, 15]

cpt = loadCPT('./IR4AVHRR6.cpt')
cpt_convert = LinearSegmentedColormap('cpt', cpt)

data = remap(files[0], bbox, 1, 'netCDF')
grid = data.ReadAsArray()

lons = np.linspace(bbox[0], bbox[2], grid.shape[1])
lats = np.linspace(bbox[1], bbox[3], grid.shape[0])

b.drawmeridians(np.linspace(bbox[0], bbox[2], 10), labels=[False, False, False, True], 
                fmt='%.2f', labelstyle='+/-', color='k')
b.drawparallels(np.linspace(bbox[1], bbox[3], 10), labels=[True, False, False, False], 
                fmt='%.2f', labelstyle='+/-', color='k')

cmap = b.imshow(grid,  origin='upper', vmin=200, vmax=300, cmap=cpt_convert)
cbar = b.colorbar(cmap, extend='both', label='Temperatura [K]')
inside = plt.axes([.17, .18, .20, .20])

inside.hist(grid.reshape(1, -1)[0], bins=100, color='k', alpha=.9)
inside.set_xlabel('Temperatura [K]', color='w')
inside.set_yticks([])
inside.tick_params(colors='w')
plt.show()